In [ ]:

import cv2
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler,EarlyStopping
from keras import backend as keras
from PIL import Image,ImageDraw
from skimage import filters,img_as_float
from skimage.io import imsave
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import xml.etree.ElementTree as ET
import os
#fig, axs = plt.subplots(1,2)
images_train={}
for dirname, _, filenames in os.walk('/kaggle/input/fracture/fracture/training'):
    for filename in filenames:
        img1=Image.open(os.path.join(dirname,filename)).convert('RGB')
        index=os.path.splitext(filename)[0]
        images_train[index]=img1
        #print(img1.size)
#axs[0].imshow(images_train['0'])

image_train_annotations={}
for dirname, _, filenames in os.walk('/kaggle/input/fracture/fracture/training_annotations'):
    for filename in filenames:
        tree=ET.parse(os.path.join(dirname,filename))
        root=tree.getroot()
        index_=os.path.splitext(filename)[0]
        img=images_train[index_]
        width,height=img.size
        #print(width,height,type(img))
        if img == None:
            del images_train[index_]
            continue
        image=Image.new(mode='1',size=(width,height),color=0)
        image1=Image.new(mode='1',size=(width,height),color=1)
        #print(type(image))
        try:
            for objects in root.iter('bndbox'):
                start1,end1=(int(objects.find('xmin').text),int(objects.find('ymin').text))
                start2,end2=(int(objects.find('xmax').text),int(objects.find('ymax').text))
                #print(start1,end1,start2,end2)
                image1=image1.crop((start1,end1,start2,end2))
                image.paste(image1,(start1,end1,start2,end2))
                #print(type(image))
            image_train_annotations[index_]=image
        except AttributeError:
            del images_train[index_]
        #print(image)
plt.imshow(image_train_annotations['1'])
        


In [ ]:

#cv expects a pointer to draw a rect
#np.copy isn't working expecting ptr<cv:Umat>
#if images_train == image_train_annotations:
#    print("doomed")
fn = lambda x : 1 if x > 240 else 0
train_data=[images_train[i] for i in sorted(images_train)]
train_annotations=[image_train_annotations[i] for i in sorted(image_train_annotations)]

fig, axs = plt.subplots(1,2)
axs[0].imshow(train_data[0])

plt.imshow(train_annotations[0])

In [ ]:
images_valid={}
for dirname, _, filenames in os.walk('/kaggle/input/fracture/fracture/validation'):
    for filename in filenames:
        img1=Image.open(os.path.join(dirname,filename)).convert('RGB')
        index=os.path.splitext(filename)[0]
        images_valid[index]=img1
#axs[0].imshow(images_train['0'])

image_valid_annotations={}

for dirname, _, filenames in os.walk('/kaggle/input/fracture/fracture/validation_annotations'):
    for filename in filenames:
        tree=ET.parse(os.path.join(dirname,filename))
        root=tree.getroot()
        index_=os.path.splitext(filename)[0]
        img=images_valid[index_]
        width,height=img.size
        #print(width,height,type(img))
        if img == None:
            del images_train[index_]
            continue
        image=Image.new(mode='1',size=(width,height),color=0)
        image1=Image.new(mode='1',size=(width,height),color=1)
        #print(type(image))
        try:
            for objects in root.iter('bndbox'):
                start1,end1=(int(objects.find('xmin').text),int(objects.find('ymin').text))
                start2,end2=(int(objects.find('xmax').text),int(objects.find('ymax').text))
                #print(start1,end1,start2,end2)
                image1=image1.crop((start1,end1,start2,end2))
                image.paste(image1,(start1,end1,start2,end2))
                #print(type(image))
            image_valid_annotations[index_]=image
        except AttributeError:
            del images_train[index_]
        #print(image)
#plt.imshow(image_valid_annotations['1'])

In [ ]:
fn = lambda x : 1 if x > 240 else 0
valid_data=[images_valid[i] for i in sorted(images_valid)]
valid_annotations=[image_valid_annotations[i] for i in sorted(image_valid_annotations)]
print(len(valid_annotations))

fig, axs = plt.subplots(1,2)
axs[0].imshow(valid_data[2])

plt.imshow(valid_annotations[2])

import tensorflow as tf

def mean_iou(self,y_true, y_pred):
        prec = []
        for t in np.arange(0.5, 1.0, 0.05):
            y_pred_ = tf.to_int32(y_pred > t)
            score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
            K.get_session().run(tf.local_variables_initializer())
            with tf.control_dependencies([up_opt]):
                score = tf.identity(score)
            prec.append(score)
        return K.mean(K.stack(prec), axis=0)

def unet(pretrained_weights = None,input_size = (128,128,1)):
    inputs = Input(input_size)
    s = Lambda(lambda x: x / 255) (inputs)

    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
    c1 = Dropout(0.1) (c1)
    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
    c2 = Dropout(0.1) (c2)
    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
    c3 = Dropout(0.2) (c3)
    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
    c4 = Dropout(0.2) (c4)
    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
    c5 = Dropout(0.3) (c5)
    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
    c6 = Dropout(0.2) (c6)
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
    c7 = Dropout(0.2) (c7)
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
    c8 = Dropout(0.1) (c8)
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
    c9 = Dropout(0.1) (c9)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

    model = Model(inputs=[inputs], outputs=[outputs])

    return(model)



import keras.preprocessing.image as image
image_data_gen = image.ImageDataGenerator(rotation_range = 40, horizontal_flip = True,
                                          vertical_flip = True, zoom_range = 0.2,
                                          shear_range = 0.2,width_shift_range = 0.2,
                                          height_shift_range = 0.2)
mask_data_gen = image.ImageDataGenerator(rotation_range = 40, horizontal_flip = True,
                                          vertical_flip = True, zoom_range = 0.2,
                                          shear_range = 0.2,width_shift_range = 0.2,
                                          height_shift_range = 0.2)
validimg_data_gen = image.ImageDataGenerator()
validmask_data_gen = image.ImageDataGenerator()


In [ ]:
current='/kaggle/working'
os.mkdir(os.path.join(current,'valid_images'))
os.mkdir(os.path.join(current,'valid_annotations'))
os.mkdir(os.path.join(current,'train_annotations'))
os.mkdir(os.path.join(current,'train_images'))
os.mkdir(os.path.join(current,'valid_images/present'))
os.mkdir(os.path.join(current,'valid_annotations/present'))
os.mkdir(os.path.join(current,'train_images/present'))
os.mkdir(os.path.join(current,'train_annotations/present'))

In [ ]:

for dirname,_,filenames in os.walk(os.path.join(current,'train_images/present')):
    for file in filenames:
        os.remove(os.path.join(dirname,file))

for dirname,_,filenames in os.walk(os.path.join(current,'train_annotations/present')):
    for file in filenames:
        os.remove(os.path.join(dirname,file))
        
for dirname,_,filenames in os.walk(os.path.join(current,'valid_images/present')):
    for file in filenames:
        os.remove(os.path.join(dirname,file))
#os.mkdir(os.path.join(current,'train_images'))
for dirname,_,filenames in os.walk(os.path.join(current,'valid_annotations/present')):
    for file in filenames:
        os.remove(os.path.join(dirname,file))





In [ ]:
!pip install -U keras_segmentation

In [ ]:
from imageio import imwrite

train_dir=os.path.join(current,'train_images/present')
train_an_dir=os.path.join(current,'train_annotations/present')
valid_dir=os.path.join(current,'valid_images/present')
valid_an_dir=os.path.join(current,'valid_annotations/present')
fn = lambda x : 1 if x > 240 else 0
for i,v in enumerate(train_data):
    if v is not None:
        cv2.imwrite(os.path.join(train_dir,str(i)+'.png'),img_as_float(np.float32(v)))

#print(type())
for i,v in enumerate(train_annotations):
    print(v)
    if v is not None:
        val = filters.threshold_otsu(np.float32(v))
        mask = np.float32(v)< val
        plt.imshow(img_as_float(mask))
        #print(img_as_float(mask))
        cv2.imwrite(os.path.join(train_an_dir,str(i)+'.png'),img_as_float(mask))
for i,v in enumerate(valid_data):
    if v is not None:
        cv2.imwrite(os.path.join(valid_dir,str(i)+'.png'),img_as_float(np.float32(v)))

for i,v in enumerate(valid_annotations):
    print(v)
    if v is not None:
        val = filters.threshold_otsu(np.float32(v))
        mask = np.float32(v)< val
        plt.imshow(mask)
        cv2.imwrite(os.path.join(valid_an_dir,str(i)+'.png'),img_as_float(mask))

valid=[]
for dirname,_,filenames in os.walk(train_an_dir):
    for file in filenames:
        valid.append(file)
for dirname,_,filenames in os.walk(train_dir):
    for file in filenames:
        if file not in valid:
            print(file)
            os.remove(os.path.join(train_dir,file))
            
            
        
    
valid=[]
for dirname,_,filenames in os.walk(valid_an_dir):
    for file in filenames:
        valid.append(file)
for dirname,_,filenames in os.walk(valid_dir):
    for file in filenames:
        if file not in valid:
            print(file)
            os.remove(os.path.join(valid_dir,file))



image_array_gen = image_data_gen.flow_from_directory(directory='/kaggle/working/train_images', class_mode = None,
                                   target_size = (128,128),color_mode='grayscale',batch_size=10)
mask_array_gen = mask_data_gen.flow_from_directory(directory='/kaggle/working/train_annotations', class_mode = None,
                                   target_size = (128,128),color_mode='grayscale',batch_size=10)
valid_image_array_gen = validimg_data_gen.flow_from_directory(directory= '/kaggle/working/valid_images', class_mode = None,
                                   target_size = (128,128),color_mode='grayscale',batch_size=10)
valid_mask_array_gen = validmask_data_gen.flow_from_directory(directory= '/kaggle/working/valid_annotations', class_mode = None,
                                   target_size = (128,128),color_mode='grayscale',batch_size=10)
train_generator = zip(image_array_gen, mask_array_gen)
valid_generator = zip(valid_image_array_gen, valid_mask_array_gen)

!pip install tf-nightly





import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
 
  try:
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    model = unet()
    model.compile(optimizer='adam', loss='binary_crossentropy' ,metrics=['binary_accuracy'])
    earlystopper = EarlyStopping(patience=5, verbose=10)
    checkpointer = ModelCheckpoint('/kaggle/working/modelunet.h5', verbose=1, save_best_only=True)
    results = model.fit_generator(train_generator, epochs=5, callbacks=[earlystopper, checkpointer])
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)



tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

with tpu_strategy.scope():
    model = unet()
    model.compile(optimizer='adam', loss='binary_crossentropy' ,metrics=['binary_accuracy'])
    earlystopper = EarlyStopping(patience=5, verbose=1)
    checkpointer = ModelCheckpoint('/kaggle/working/modelunet.h5', verbose=1, save_best_only=True)
results = model.fit_generator(train_generator, epochs=5, 
                        callbacks=[earlystopper, checkpointer])

model = unet()
model.compile(optimizer='adam', loss='binary_crossentropy' ,metrics=['binary_accuracy'])
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('/kaggle/working/modelunet.h5', verbose=1, save_best_only=True)
results = model.fit_generator(train_generator, epochs=40, steps_per_epoch=100,
                        callbacks=[earlystopper, checkpointer])

from skimage.transform import resize
def prediction(imagePath):
    img=cv2.imread(imagePath)
    x_test= np.zeros((1, 128,128,1), dtype=np.uint8)
    img=resize(img,(128,128,1),mode='constant',preserve_range=True)
    x_test[0]=img
    preds_test= model.predict(x_test, verbose=1)
    #preds_test = (preds_test > 0.5).astype(np.uint8)
    mask=preds_test[0]
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            if mask[i][j] == 1:
                mask[i][j] = 255
            else:
                mask[i][j] = 0
    merged_image = cv2.merge((mask,mask,mask))
    print(preds_test[1])
    plt.imshow(merged_image)

prediction('/kaggle/working/valid_images/present/2.png')
#plt.imshow(PIL.Image.open('/kaggle/working/valid_images/present/2.png'))

i=Image.open('/kaggle/working/train_annotations/present/40.tiff')
plt.imshow(i.convert('L').point(fn,mode='1'))

In [ ]:
from keras_segmentation.models.unet import resnet50_unet as res_unet

model = res_unet(n_classes=2 ,  input_height=256, input_width=256  )
model.train( 
    train_images =  train_dir,
    train_annotations = train_an_dir,
    checkpoints_path = current+'/experimodel',
   epochs=9
)

In [ ]:
out = model.predict_segmentation(
	inp=valid_dir+"/3.png", 
	out_fname=valid_dir+"/predict/output.png" 
)

In [ ]:
plt.imshow(out)

In [ ]:
plt.imshow(Image.open(valid_dir+"/3.png"))

In [ ]:
plt.imshow(Image.open(valid_an_dir+"/3.png"))